# Heaps / Priority Queue

Solutions to Leetcode problems that use heaps or priority queues.

> Anytime you see *k most* or *k least*, you want to use a heap. PQs use heaps and are very important for graph algorithms.

In [1]:
from typing import List

# Problem One: Kth Largest Element in a Stream (Easy)

[Leetcode #703](https://leetcode.com/problems/kth-largest-element-in-a-stream/)

- Use min heap of size k. Notice that we never delete items. So once we are given numbers, only store the k biggest in a min heap. When we add, we remove min if we are bigger than k. This way min is always kth largest

In [2]:
class KthLargest:

    def __init__(self, k: int, nums: List[int]):
        self.k = k
        self.minHeap = nums
        heapq.heapify(self.minHeap)
        while len(self.minHeap) > k:
            heapq.heappop(self.minHeap)
        

    def add(self, val: int) -> int:
        heapq.heappush(self.minHeap, val)
        if len(self.minHeap) > self.k:
            heapq.heappop(self.minHeap)
        return self.minHeap[0]

# Problem Two: Last Stone Weight (Easy)

[Leetcode #1046](https://leetcode.com/problems/last-stone-weight/)

- We will be using a min heap, so negate all weights
- Pop two "min" value stones
    - If they are different weight:
        - Push the difference of their weight back to the heap. Make sure it is a negative value.
    - If they are same weight we don't need to do anything
- Return the negative of the first value in stones, or `0` if there is nothing left in stones

In [3]:
def lastStoneWeight(stones: List[int]) -> int:
        # Negate weights since we have access to minHeap
        stones = [-s for s in stones]
        heapq.heapify(stones)  # O(NlogN)

        while len(stones) > 1:
            first, second = heapq.heappop(stones), heapq.heappop(stones)

            # Remember the weight are negated since we can only use minHeap
            if second > first:
                heapq.heappush(stones, first - second)  # Add the remaining weight of the stone
        
        return -stones[0] if stones else 0

# Problem Three: K Closest Points to Origin (Medium)

[Leetcode #973](https://leetcode.com/problems/k-closest-points-to-origin/)

- Create min heap containing `(dist, x, y)` sorted by `dist`
- Pop the first `k` and append `(x, y)` values to result

In [4]:
def kClosest(points: List[List[int]], k: int) -> List[List[int]]:
        dist = lambda a, b: sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
        distances = [ (dist((x, y), (0, 0)), x, y) for x, y in points]
        heapq.heapify(distances)

        res = []

        for i in range(k):
            d, x, y = heapq.heappop(distances)
            res.append([x, y])

        return res

# Problem Four: Kth Largest Element in an Array (Medium)

[Leetcode #215](https://leetcode.com/problems/kth-largest-element-in-an-array/)

- Negate values since we will use a python min heap
- Build heap from `nums`
- Pop heap `k` times and return the `kth` pop (don't forget to negate this return value)

In [5]:
def findKthLargest(nums: List[int], k: int) -> int:
        nums = [-n for n in nums]
        heapq.heapify(nums)

        last = None
        for i in range(k):
            last = heapq.heappop(nums)
        
        return -last

# Problem Five: Task Scheduler (Medium)

[Leetcode #621](https://leetcode.com/problems/task-scheduler/)

- Create hashmap of counts of each task
- Create a **Max Heap** of remaining counts. We always want to run the task with most remaining count
- Create a queue of idled tasks as `(count, availableTime)` pairs where `availableTime` is the next time it can be run again
- Initialize timer to 0

- While the heap or the queue are full:
    - Increment timer
    - If max heap is full:
        - Pop heap
        - Decrement from the count
        - If it is nonzero, add `(count, time + n)` to queue
    - If queue is nonempty and first element is done idling, push it back into heap

- Return timer

In [7]:
def leastInterval(tasks: List[str], n: int) -> int:
        # Build dict of counts of tasks
        taskCount = Counter(tasks)
        # Negate values so we can use max heap from min heap
        maxHeap = [-count for count in taskCount.values()]
        heapq.heapify(maxHeap)  # Build max heap -> N*log(26) = O(N) time complexity
        # Current time
        time = 0
        # Queue of (count, availableTime) pairs
        q = collections.deque()

        while maxHeap or q:
            time += 1

            if maxHeap:
                # Pop task with most occurunces left. Reduce magnitude of occurences by one.
                count = 1 + heapq.heappop(maxHeap)

                if count:
                    q.append([count, time + n])
            
            # If queue is non empty and first element is now free
            if q and q[0][1] == time:
                heapq.heappush(maxHeap, q.popleft()[0])
        
        return time

# Problem Six: Design Twitter (Medium but should be Hard)

[Leetcode #355](https://leetcode.com/problems/design-twitter/)

- Basically merging `k` sorted lists of tweets using min heap

- `follow()`/`unfollow()`:
    - Keep a HashMap which maps each user to a HashSet of users they follow
    - For follow, append the `followeeId` to HashSet of user
     - For unfollow, remove the `followeeId` from HashSet of user IF already existed

- `postTweet`:
    - Keep a HashMap which maps each user to a List of their tweets. Each entry in the list is a tuple `(time, tweetId)`
    - Increment the timer

- `getNewsFeed`:
    - Initialize empty heap and result array
    - Don't forget to add user to their own following list since the news feed must have users own posts too
    - For each followee, add their most recent tweet to the heap. Heap entries are tuples `(time, tweetId, followeeId, nextTweetIndex)`. The `time` is what is used for ordering the heap. `followeeId` and `nextTweetIndex` are used by the algorithm to retrieve the users next tweet when this one is popped from the heap.
    - While the heap is full and the length of result array is less than ten:
        - Pop from heap, add to result array
        - If user has another tweet, retrieve it and it to heap
    - Return result array

In [9]:
class Twitter:

    def __init__(self):
        self.time = 0
        self.tweetMap = defaultdict(list)
        self.followMap = defaultdict(set)
        

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweetMap[userId].append([self.time, tweetId])
        self.time -= 1

    def getNewsFeed(self, userId: int) -> List[int]:
        res = []  # List of 10 tweets retrieved
        minHeap = []

        self.followMap[userId].add(userId)
        for followeeId in self.followMap[userId]:
            if followeeId in self.tweetMap:
                index = len(self.tweetMap[followeeId]) - 1
                time, tweetId = self.tweetMap[followeeId][index]
                # Add time and tweetId to minHeap
                # Also add followeeId and the index of this followees twwets for retrieving the next tweet
                minHeap.append([time, tweetId, followeeId, index - 1])
        
        heapq.heapify(minHeap)  # Create heap
        while minHeap and len(res) < 10:
            # Get most recent tweet
            time, tweetId, followeeId, index = heapq.heappop(minHeap)
            res.append(tweetId)
            
            # If user has any tweets left
            if index >= 0:
                # Get next tweet
                time, tweetId = self.tweetMap[followeeId][index]
                # Add next tweet to heap
                heapq.heappush(minHeap, [time, tweetId, followeeId, index - 1])
        
        return res


    def follow(self, followerId: int, followeeId: int) -> None:
        self.followMap[followerId].add(followeeId)
        

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followeeId in self.followMap[followerId]:
            self.followMap[followerId].remove(followeeId)


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)

# Problem Seven: Find Median From Data Stream (*Hard*)

[Leetcode #295](https://leetcode.com/problems/find-median-from-data-stream/)

- Maintain two heaps, `small` and `large` such that:
    - `small` is a Max Heap, `large` is a Min Heap
    - Every number in `small` is smaller than every number in `large`
    - The size of `small` and size of `larger` differ by no more than one

- To get the median:
     - If `small` is larger in size, return `getMax()` on the max heap
     - If `large` is larger in size, return `getMin()` on the min heap
     - If they are the same size, return `(small.getMax() + large.getMin()) / 2`

In [10]:
class MedianFinder:

    def __init__(self):
        self.large, self.small = [], []
        

    def addNum(self, num: int) -> None:
        # Push num to small heap
        heapq.heappush(self.small, -1 * num)

        # Make sure every element in small heap is <= every num in large heap
        if self.small and self.large and (-1 * self.small[0] > self.large[0]):
            val = -1 * heapq.heappop(self.small)  # Make sure to undo the negation from small heap
            heapq.heappush(self.large, val)
        
        # Are the sizes uneven
        if len(self.small) > len(self.large) + 1:
            val = -1 * heapq.heappop(self.small)  # Make sure to undo the negation from small heap
            heapq.heappush(self.large, val)
        
        if len(self.large) > len(self.small) + 1:
            val = heapq.heappop(self.large)
            heapq.heappush(self.small, -1  * val)  # Make sure to negate the value from the large heap
        

    def findMedian(self) -> float:
        if len(self.small) > len(self.large):
            return -self.small[0]
        if len(self.large) > len(self.small):
            return self.large[0]
        
        return (self.large[0] + -self.small[0]) / 2
        


# Your MedianFinder object will be instantiated and called as such:
# obj = MedianFinder()
# obj.addNum(num)
# param_2 = obj.findMedian()